# Implementación de un KD-Tree con Identificadores Enteros y Manejo de Listas Desiguales

Este notebook presenta la implementación de un KD-Tree que permite manejar listas con nodos definidos por un entero identificador y tres coordenadas (doubles). También explora casos con listas desiguales (diferentes tamaños).

### Representación y Modificaciones
* Cada nodo ahora se define como una lista simple: `ID, x, y, z`.
* Ejemplo: `[1, 1.1, 2.2, 3.3]` indica que el nodo tiene ID `1` y coordenadas `(1.1, 2.2, 3.3)`.
* Esto elimina la necesidad de una clase separada para representar nodos.

In [1]:
# Importar bibliotecas necesarias
import numpy as np
from scipy.spatial import KDTree

# Función para construir un KD-Tree a partir de una lista de nodos
def create_kdtree(points):
    """
    Crear un KD-Tree utilizando las coordenadas de una lista de nodos.
    Cada nodo tiene la forma [ID, x, y, z].
    """
    # Extraer sólo las coordenadas (x, y, z) de los puntos
    coordinates = [point[1:] for point in points]
    # Crear el KD-Tree
    tree = KDTree(coordinates)
    return tree

# Función de emparejamiento inyectivo
def nearest_neighbor_matching(points_a, points_b):
    """
    Encuentra los vecinos más cercanos de forma inyectiva (sin reutilización de nodos).
    Si las listas son desiguales, algunos nodos de la lista más grande quedarán sin asociar.
    """
    # Crear KD-Tree utilizando la lista 'points_b'
    tree_b = create_kdtree(points_b)
    
    # Conjunto para rastrear los IDs ya usados
    used_ids = set()
    matches = []  # Lista final de emparejamientos

    # Iterar sobre los puntos de la lista 'points_a'
    for point_a in points_a:
        # Realizar la búsqueda de vecinos más cercanos en 'points_b'
        distances, indices = tree_b.query(point_a[1:], k=len(points_b))

        # Buscar el vecino no usado más cercano
        for index in indices:
            neighbor = points_b[index]
            if neighbor[0] not in used_ids:  # Verificar que el ID no está reutilizado
                matches.append((point_a, neighbor))  # Emparejar los nodos
                used_ids.add(neighbor[0])
                break

    return matches

# Función para imprimir resultados del emparejamiento
def print_matches(matches):
    """
    Imprime las asociaciones entre nodos tras el emparejamiento.
    """
    for match in matches:
        point_a, point_b = match
        print(f"Nodo {point_a[0]} ({point_a[1:]}) -> Nodo {point_b[0]} ({point_b[1:]})")

    print("\nNodos emparejados: ", len(matches))


# Ejemplo 1: Listas de Tamaño Igual

En este ejemplo, las dos listas tienen el mismo número de nodos.

In [2]:
# Lista A (nodos 1x)
points_a = [
    [11, 1.0, 2.0, 3.0],
    [12, 4.0, 5.0, 6.0],
    [13, 7.0, 8.0, 9.0]
]

# Lista B (nodos 2x)
points_b = [
    [21, 1.1, 2.2, 3.3],
    [22, 4.1, 5.1, 6.1],
    [23, 7.1, 8.1, 9.1]
]

# Realizar el emparejamiento
matches = nearest_neighbor_matching(points_a, points_b)

# Imprimir resultados
print("Emparejamiento para listas de tamaño igual:\n")
print_matches(matches)


Emparejamiento para listas de tamaño igual:

Nodo 11 ([1.0, 2.0, 3.0]) -> Nodo 21 ([1.1, 2.2, 3.3])
Nodo 12 ([4.0, 5.0, 6.0]) -> Nodo 22 ([4.1, 5.1, 6.1])
Nodo 13 ([7.0, 8.0, 9.0]) -> Nodo 23 ([7.1, 8.1, 9.1])

Nodos emparejados:  3


# Ejemplo 2: Listas de Tamaño Desigual

En este caso, la lista `A` tiene más nodos que la lista `B`.

In [ ]:
# Lista A
points_a = [
    [11, 1.0, 2.0, 3.0],
    [12, 4.0, 5.0, 6.0],
    [13, 7.0, 8.0, 9.0],
    [14, 10.0, 11.0, 12.0]
]

# Lista B
points_b = [
    [21, 1.1, 2.2, 3.3],
    [22, 4.1, 5.1, 6.1],
    [23, 7.1, 8.1, 9.1]
]

# Realizar el emparejamiento
matches = nearest_neighbor_matching(points_a, points_b)

# Imprimir resultados
print("Emparejamiento para listas de tamaño desigual:\n")
print_matches(matches)


Emparejamiento para listas de tamaño desigual:

Nodo 11 ([1.0, 2.0, 3.0]) -> Nodo 21 ([1.1, 2.2, 3.3])
Nodo 12 ([4.0, 5.0, 6.0]) -> Nodo 22 ([4.1, 5.1, 6.1])
Nodo 13 ([7.0, 8.0, 9.0]) -> Nodo 23 ([7.1, 8.1, 9.1])

Nodos emparejados:  3


# Observaciones
1. **Tamaño desigual:** Cuando las listas son desiguales, algunos nodos de la lista más grande (en este caso `A`) no obtendrán pareja.
2. **Nodos identificados por enteros:** Los IDs únicos hacen que los nodos sean fáciles de rastrear.
3. **Emparejamiento inyectivo:** La implementación garantiza que cada nodo de `B` solo sea asociado una vez.
4. **Eficiencia:** Gracias al KD-Tree, el emparejamiento es eficiente incluso para listas grandes.